In [1]:
#Importing Lib
import numpy as np
import pandas as pd 

#Importing dataset
train = pd.read_csv('C:\\Users\\hp-pc\\Desktop\\Deep Summer\\Titanic\\train.csv')
test = pd.read_csv('C:\\Users\\hp-pc\\Desktop\\Deep Summer\\Titanic\\test.csv')
full_data = [train, test]
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
#Family Size new feature 
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
for dataset in full_data: 
    dataset['Age']= dataset['Age'].fillna(dataset['Age'].dropna().mean())
    dataset['Fare']= dataset['Fare'].fillna(dataset['Fare'].dropna().mean())
    dataset['Embarked']= dataset['Embarked'].fillna(value='S')
    dataset['Sex']= dataset['Sex'].fillna(value='male')
for dataset in full_data:   
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
for dataset in full_data:     
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 
for dataset in full_data:    
    dataset['Sex'] = dataset['Sex'].map( {'male': 0, 'female': 1} ).astype(int)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
# Define function to extract titles from passenger names
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [2]:
X_train = train.iloc[:,[2,4,5,6,7,9,11,12,13,14,15,16]].values
y_train = train.iloc[:,1].values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
labelencoder_X1 = LabelEncoder()
X_train[:,11] =  labelencoder_X1.fit_transform(X_train[:,11])
onehotencoder = OneHotEncoder(categorical_features = [11])
X_train = onehotencoder.fit_transform(X_train).toarray()
X_train = X_train[:,1:]
onehotencoder_2 = OneHotEncoder(categorical_features = [10])
X_train = onehotencoder_2.fit_transform(X_train).toarray()
X_train = X_train[:,1:]
onehotencoder_3 = OneHotEncoder(categorical_features = [6])
X_train = onehotencoder_3.fit_transform(X_train).toarray()
X_train = X_train[:,1:]
from sklearn.preprocessing import StandardScaler 
sc= StandardScaler()
X_train=sc.fit_transform(X_train)

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = 17))
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 300],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
801/801 [==============================] - 1s 2ms/step - loss: 0.6914 - acc: 0.6167
Epoch 2/100
801/801 [==============================] - 0s 132us/step - loss: 0.6848 - acc: 0.6217
Epoch 3/100
801/801 [==============================] - 0s 123us/step - loss: 0.6468 - acc: 0.6217
Epoch 4/100
801/801 [==============================] - 0s 100us/step - loss: 0.5645 - acc: 0.6217
Epoch 5/100
801/801 [==============================] - 0s 100us/step - loss: 0.5274 - acc: 0.6217
Epoch 6/100
801/801 [==============================] - 0s 103us/step - loss: 0.5186 - acc: 0.6217
Epoch 7/100
801/801 [==============================] - 0s 97us/step - loss: 0.5120 - acc: 0.6217
Epoch 8/100
801/801 [==============================] - 0s 98us/step - loss: 0.5052 - acc: 0.7553
Epoch 9/100
801/801 [==============================] - 0s 108us/step - loss: 0.5023 - acc: 0.8265
Epoch 10/100
801/801 [==============================] - 0s 100us/step - loss: 0.4968 - acc: 0.8265
Epoch 11/100
801/801 [=

Epoch 84/100
801/801 [==============================] - 0s 164us/step - loss: 0.3798 - acc: 0.8564
Epoch 85/100
801/801 [==============================] - 0s 135us/step - loss: 0.3789 - acc: 0.8564 0s - loss: 0.3675 - acc: 0.860
Epoch 86/100
801/801 [==============================] - 0s 90us/step - loss: 0.3779 - acc: 0.8577
Epoch 87/100
801/801 [==============================] - 0s 72us/step - loss: 0.3776 - acc: 0.8602
Epoch 88/100
801/801 [==============================] - 0s 73us/step - loss: 0.3762 - acc: 0.8589
Epoch 89/100
801/801 [==============================] - 0s 84us/step - loss: 0.3758 - acc: 0.8589
Epoch 90/100
801/801 [==============================] - 0s 83us/step - loss: 0.3777 - acc: 0.8627
Epoch 91/100
801/801 [==============================] - 0s 74us/step - loss: 0.3761 - acc: 0.8614
Epoch 92/100
801/801 [==============================] - 0s 65us/step - loss: 0.3750 - acc: 0.8614
Epoch 93/100
801/801 [==============================] - 0s 62us/step - loss: 0.3726 -

802/802 [==============================] - 0s 72us/step - loss: 0.3764 - acc: 0.8416
Epoch 67/100
802/802 [==============================] - 0s 73us/step - loss: 0.3767 - acc: 0.8354
Epoch 68/100
802/802 [==============================] - 0s 62us/step - loss: 0.3765 - acc: 0.8416
Epoch 69/100
802/802 [==============================] - 0s 64us/step - loss: 0.3754 - acc: 0.8404
Epoch 70/100
802/802 [==============================] - 0s 68us/step - loss: 0.3753 - acc: 0.8416
Epoch 71/100
802/802 [==============================] - 0s 65us/step - loss: 0.3757 - acc: 0.8429
Epoch 72/100
802/802 [==============================] - 0s 60us/step - loss: 0.3760 - acc: 0.8392
Epoch 73/100
802/802 [==============================] - 0s 62us/step - loss: 0.3739 - acc: 0.8454
Epoch 74/100
802/802 [==============================] - 0s 61us/step - loss: 0.3763 - acc: 0.8441
Epoch 75/100
802/802 [==============================] - 0s 62us/step - loss: 0.3737 - acc: 0.8454
Epoch 76/100
802/802 [===========

802/802 [==============================] - 0s 62us/step - loss: 0.3692 - acc: 0.8516
Epoch 50/100
802/802 [==============================] - 0s 62us/step - loss: 0.3686 - acc: 0.8516
Epoch 51/100
802/802 [==============================] - 0s 62us/step - loss: 0.3686 - acc: 0.8504
Epoch 52/100
802/802 [==============================] - 0s 60us/step - loss: 0.3681 - acc: 0.8566
Epoch 53/100
802/802 [==============================] - 0s 62us/step - loss: 0.3678 - acc: 0.8479
Epoch 54/100
802/802 [==============================] - 0s 60us/step - loss: 0.3691 - acc: 0.8504
Epoch 55/100
802/802 [==============================] - 0s 63us/step - loss: 0.3667 - acc: 0.8504
Epoch 56/100
802/802 [==============================] - 0s 66us/step - loss: 0.3666 - acc: 0.8504
Epoch 57/100
802/802 [==============================] - 0s 61us/step - loss: 0.3657 - acc: 0.8516
Epoch 58/100
802/802 [==============================] - 0s 61us/step - loss: 0.3652 - acc: 0.8541
Epoch 59/100
802/802 [===========

802/802 [==============================] - 0s 60us/step - loss: 0.3984 - acc: 0.8329
Epoch 33/100
802/802 [==============================] - 0s 63us/step - loss: 0.3962 - acc: 0.8329
Epoch 34/100
802/802 [==============================] - 0s 60us/step - loss: 0.3950 - acc: 0.8354
Epoch 35/100
802/802 [==============================] - 0s 65us/step - loss: 0.3919 - acc: 0.8354
Epoch 36/100
802/802 [==============================] - 0s 62us/step - loss: 0.3894 - acc: 0.8354
Epoch 37/100
802/802 [==============================] - 0s 62us/step - loss: 0.3890 - acc: 0.8429
Epoch 38/100
802/802 [==============================] - 0s 62us/step - loss: 0.3864 - acc: 0.8392
Epoch 39/100
802/802 [==============================] - 0s 61us/step - loss: 0.3851 - acc: 0.8441
Epoch 40/100
802/802 [==============================] - 0s 61us/step - loss: 0.3842 - acc: 0.8416
Epoch 41/100
802/802 [==============================] - 0s 61us/step - loss: 0.3826 - acc: 0.8416
Epoch 42/100
802/802 [===========

...

802/802 [==============================] - 0s 98us/step - loss: 0.3892 - acc: 0.8404
Epoch 57/300
802/802 [==============================] - 0s 94us/step - loss: 0.3879 - acc: 0.8379
Epoch 58/300
802/802 [==============================] - 0s 94us/step - loss: 0.3879 - acc: 0.8404
Epoch 59/300
802/802 [==============================] - 0s 96us/step - loss: 0.3872 - acc: 0.8392
Epoch 60/300
802/802 [==============================] - 0s 95us/step - loss: 0.3867 - acc: 0.8367
Epoch 61/300
802/802 [==============================] - 0s 96us/step - loss: 0.3868 - acc: 0.8379
Epoch 62/300
802/802 [==============================] - 0s 95us/step - loss: 0.3867 - acc: 0.8392
Epoch 63/300
802/802 [==============================] - 0s 95us/step - loss: 0.3860 - acc: 0.8392
Epoch 64/300
802/802 [==============================] - 0s 95us/step - loss: 0.3849 - acc: 0.8392
Epoch 65/300
802/802 [==============================] - 0s 95us/step - loss: 0.3850 - acc: 0.8367
Epoch 66/300
802/802 [===========

802/802 [==============================] - 0s 94us/step - loss: 0.3622 - acc: 0.8603
Epoch 140/300
802/802 [==============================] - 0s 93us/step - loss: 0.3626 - acc: 0.8653
Epoch 141/300
802/802 [==============================] - 0s 93us/step - loss: 0.3624 - acc: 0.8641
Epoch 142/300
802/802 [==============================] - 0s 96us/step - loss: 0.3622 - acc: 0.8616
Epoch 143/300
802/802 [==============================] - 0s 95us/step - loss: 0.3624 - acc: 0.8591
Epoch 144/300
802/802 [==============================] - 0s 96us/step - loss: 0.3610 - acc: 0.8653
Epoch 145/300
802/802 [==============================] - 0s 90us/step - loss: 0.3614 - acc: 0.8653
Epoch 146/300
802/802 [==============================] - 0s 92us/step - loss: 0.3612 - acc: 0.8678
Epoch 147/300
802/802 [==============================] - 0s 96us/step - loss: 0.3615 - acc: 0.8653
Epoch 148/300
802/802 [==============================] - 0s 96us/step - loss: 0.3606 - acc: 0.8628
Epoch 149/300
802/802 [=

802/802 [==============================] - 0s 118us/step - loss: 0.3488 - acc: 0.8666
Epoch 222/300
802/802 [==============================] - 0s 106us/step - loss: 0.3488 - acc: 0.8616
Epoch 223/300
802/802 [==============================] - 0s 103us/step - loss: 0.3476 - acc: 0.8666
Epoch 224/300
802/802 [==============================] - 0s 100us/step - loss: 0.3481 - acc: 0.8691
Epoch 225/300
802/802 [==============================] - 0s 103us/step - loss: 0.3485 - acc: 0.8678
Epoch 226/300
802/802 [==============================] - 0s 109us/step - loss: 0.3475 - acc: 0.8666
Epoch 227/300
802/802 [==============================] - 0s 108us/step - loss: 0.3480 - acc: 0.8691
Epoch 228/300
802/802 [==============================] - 0s 105us/step - loss: 0.3471 - acc: 0.8653
Epoch 229/300
802/802 [==============================] - 0s 128us/step - loss: 0.3468 - acc: 0.8641
Epoch 230/300
802/802 [==============================] - 0s 118us/step - loss: 0.3470 - acc: 0.8641
Epoch 231/300


891/891 [==============================] - 0s 108us/step - loss: 0.6671 - acc: 0.6162
Epoch 4/100
891/891 [==============================] - 0s 109us/step - loss: 0.6378 - acc: 0.6162
Epoch 5/100
891/891 [==============================] - 0s 114us/step - loss: 0.5952 - acc: 0.6319
Epoch 6/100
891/891 [==============================] - 0s 113us/step - loss: 0.5510 - acc: 0.7823
Epoch 7/100
891/891 [==============================] - 0s 138us/step - loss: 0.5192 - acc: 0.8182
Epoch 8/100
891/891 [==============================] - 0s 133us/step - loss: 0.4983 - acc: 0.8193
Epoch 9/100
891/891 [==============================] - 0s 101us/step - loss: 0.4807 - acc: 0.8238
Epoch 10/100
891/891 [==============================] - 0s 102us/step - loss: 0.4640 - acc: 0.8193
Epoch 11/100
891/891 [==============================] - 0s 123us/step - loss: 0.4493 - acc: 0.8238
Epoch 12/100
891/891 [==============================] - 0s 133us/step - loss: 0.4373 - acc: 0.8238
Epoch 13/100
891/891 [=======

891/891 [==============================] - 0s 92us/step - loss: 0.3705 - acc: 0.8496
Epoch 87/100
891/891 [==============================] - 0s 94us/step - loss: 0.3704 - acc: 0.8496
Epoch 88/100
891/891 [==============================] - 0s 98us/step - loss: 0.3701 - acc: 0.8496
Epoch 89/100
891/891 [==============================] - 0s 112us/step - loss: 0.3701 - acc: 0.8507
Epoch 90/100
891/891 [==============================] - 0s 90us/step - loss: 0.3696 - acc: 0.8519
Epoch 91/100
891/891 [==============================] - 0s 89us/step - loss: 0.3697 - acc: 0.8451
Epoch 92/100
891/891 [==============================] - 0s 111us/step - loss: 0.3694 - acc: 0.8462
Epoch 93/100
891/891 [==============================] - 0s 110us/step - loss: 0.3693 - acc: 0.8496
Epoch 94/100
891/891 [==============================] - 0s 90us/step - loss: 0.3693 - acc: 0.8474
Epoch 95/100
891/891 [==============================] - 0s 115us/step - loss: 0.3683 - acc: 0.8507
Epoch 96/100
891/891 [=======

In [10]:
print (best_parameters,best_accuracy)

{'batch_size': 32, 'epochs': 100, 'optimizer': 'rmsprop'} 0.827160493827


In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = 17))
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 32, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = 1)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/100
801/801 [==============================] - 4s 4ms/step - loss: 0.6895 - acc: 0.6192
Epoch 2/100
801/801 [==============================] - 0s 122us/step - loss: 0.6754 - acc: 0.6217
Epoch 3/100
801/801 [==============================] - 0s 124us/step - loss: 0.6402 - acc: 0.6217
Epoch 4/100
801/801 [==============================] - 0s 131us/step - loss: 0.5931 - acc: 0.6217
Epoch 5/100
801/801 [==============================] - 0s 126us/step - loss: 0.5534 - acc: 0.6217
Epoch 6/100
801/801 [==============================] - 0s 153us/step - loss: 0.5300 - acc: 0.7041
Epoch 7/100
801/801 [==============================] - 0s 125us/step - loss: 0.5203 - acc: 0.8065
Epoch 8/100
801/801 [==============================] - 0s 126us/step - loss: 0.5129 - acc: 0.8140
Epoch 9/100
801/801 [==============================] - 0s 139us/step - loss: 0.5061 - acc: 0.8240
Epoch 10/100
801/801 [==============================] - 0s 128us/step - loss: 0.4990 - acc: 0.8302
Epoch 11/100
801/801 

801/801 [==============================] - 0s 143us/step - loss: 0.3644 - acc: 0.8564
Epoch 84/100
801/801 [==============================] - 0s 115us/step - loss: 0.3641 - acc: 0.8539
Epoch 85/100
801/801 [==============================] - 0s 121us/step - loss: 0.3635 - acc: 0.8602
Epoch 86/100
801/801 [==============================] - 0s 120us/step - loss: 0.3621 - acc: 0.8539
Epoch 87/100
801/801 [==============================] - 0s 114us/step - loss: 0.3620 - acc: 0.8602
Epoch 88/100
801/801 [==============================] - 0s 123us/step - loss: 0.3612 - acc: 0.8527
Epoch 89/100
801/801 [==============================] - 0s 128us/step - loss: 0.3612 - acc: 0.8602
Epoch 90/100
801/801 [==============================] - 0s 127us/step - loss: 0.3617 - acc: 0.8552
Epoch 91/100
801/801 [==============================] - 0s 128us/step - loss: 0.3603 - acc: 0.8614
Epoch 92/100
801/801 [==============================] - 0s 121us/step - loss: 0.3604 - acc: 0.8602
Epoch 93/100
801/801 [=

802/802 [==============================] - 0s 101us/step - loss: 0.3842 - acc: 0.8367
Epoch 66/100
802/802 [==============================] - 0s 105us/step - loss: 0.3830 - acc: 0.8404
Epoch 67/100
802/802 [==============================] - 0s 103us/step - loss: 0.3832 - acc: 0.8392
Epoch 68/100
802/802 [==============================] - 0s 99us/step - loss: 0.3835 - acc: 0.8379
Epoch 69/100
802/802 [==============================] - 0s 106us/step - loss: 0.3828 - acc: 0.8454
Epoch 70/100
802/802 [==============================] - 0s 96us/step - loss: 0.3820 - acc: 0.8392
Epoch 71/100
802/802 [==============================] - 0s 100us/step - loss: 0.3818 - acc: 0.8367
Epoch 72/100
802/802 [==============================] - 0s 100us/step - loss: 0.3814 - acc: 0.8441
Epoch 73/100
802/802 [==============================] - 0s 112us/step - loss: 0.3808 - acc: 0.8379
Epoch 74/100
802/802 [==============================] - 0s 105us/step - loss: 0.3804 - acc: 0.8392
Epoch 75/100
802/802 [===

802/802 [==============================] - 0s 110us/step - loss: 0.3773 - acc: 0.8466
Epoch 48/100
802/802 [==============================] - 0s 103us/step - loss: 0.3765 - acc: 0.8479
Epoch 49/100
802/802 [==============================] - 0s 238us/step - loss: 0.3766 - acc: 0.8479
Epoch 50/100
802/802 [==============================] - 0s 200us/step - loss: 0.3760 - acc: 0.8479
Epoch 51/100
802/802 [==============================] - 0s 218us/step - loss: 0.3752 - acc: 0.8491
Epoch 52/100
802/802 [==============================] - 0s 191us/step - loss: 0.3747 - acc: 0.8529
Epoch 53/100
802/802 [==============================] - 0s 154us/step - loss: 0.3741 - acc: 0.8479
Epoch 54/100
802/802 [==============================] - 0s 124us/step - loss: 0.3734 - acc: 0.8504
Epoch 55/100
802/802 [==============================] - 0s 227us/step - loss: 0.3728 - acc: 0.8516
Epoch 56/100
802/802 [==============================] - 0s 160us/step - loss: 0.3728 - acc: 0.8516
Epoch 57/100
802/802 [=

802/802 [==============================] - 0s 143us/step - loss: 0.3920 - acc: 0.8342 0s - loss: 0.4178 - acc: 0.817
Epoch 29/100
802/802 [==============================] - 0s 195us/step - loss: 0.3912 - acc: 0.8416
Epoch 30/100
802/802 [==============================] - 0s 165us/step - loss: 0.3905 - acc: 0.8392
Epoch 31/100
802/802 [==============================] - 0s 147us/step - loss: 0.3898 - acc: 0.8379
Epoch 32/100
802/802 [==============================] - 0s 125us/step - loss: 0.3892 - acc: 0.8392
Epoch 33/100
802/802 [==============================] - 0s 131us/step - loss: 0.3884 - acc: 0.8367
Epoch 34/100
802/802 [==============================] - 0s 118us/step - loss: 0.3883 - acc: 0.8379
Epoch 35/100
802/802 [==============================] - 0s 153us/step - loss: 0.3869 - acc: 0.8354
Epoch 36/100
802/802 [==============================] - 0s 180us/step - loss: 0.3863 - acc: 0.8404
Epoch 37/100
802/802 [==============================] - 0s 134us/step - loss: 0.3855 - acc:

...

802/802 [==============================] - 0s 150us/step - loss: 0.3785 - acc: 0.8504
Epoch 57/100
802/802 [==============================] - 0s 126us/step - loss: 0.3780 - acc: 0.8491
Epoch 58/100
802/802 [==============================] - 0s 162us/step - loss: 0.3770 - acc: 0.8479
Epoch 59/100
802/802 [==============================] - 0s 116us/step - loss: 0.3770 - acc: 0.8479
Epoch 60/100
802/802 [==============================] - 0s 182us/step - loss: 0.3769 - acc: 0.8479
Epoch 61/100
802/802 [==============================] - 0s 131us/step - loss: 0.3770 - acc: 0.8466
Epoch 62/100
802/802 [==============================] - 0s 122us/step - loss: 0.3764 - acc: 0.8504
Epoch 63/100
802/802 [==============================] - 0s 152us/step - loss: 0.3758 - acc: 0.8491
Epoch 64/100
802/802 [==============================] - 0s 137us/step - loss: 0.3757 - acc: 0.8491
Epoch 65/100
802/802 [==============================] - 0s 126us/step - loss: 0.3744 - acc: 0.8504
Epoch 66/100
802/802 [=

802/802 [==============================] - 0s 126us/step - loss: 0.3854 - acc: 0.8429
Epoch 39/100
802/802 [==============================] - 0s 111us/step - loss: 0.3850 - acc: 0.8354
Epoch 40/100
802/802 [==============================] - 0s 157us/step - loss: 0.3837 - acc: 0.8416
Epoch 41/100
802/802 [==============================] - 0s 148us/step - loss: 0.3828 - acc: 0.8342
Epoch 42/100
802/802 [==============================] - 0s 152us/step - loss: 0.3821 - acc: 0.8354
Epoch 43/100
802/802 [==============================] - 0s 136us/step - loss: 0.3814 - acc: 0.8342
Epoch 44/100
802/802 [==============================] - 0s 154us/step - loss: 0.3811 - acc: 0.8416
Epoch 45/100
802/802 [==============================] - 0s 157us/step - loss: 0.3803 - acc: 0.8367
Epoch 46/100
802/802 [==============================] - 0s 151us/step - loss: 0.3799 - acc: 0.8416
Epoch 47/100
802/802 [==============================] - 0s 158us/step - loss: 0.3790 - acc: 0.8392
Epoch 48/100
802/802 [=

802/802 [==============================] - 0s 133us/step - loss: 0.4273 - acc: 0.8267
Epoch 21/100
802/802 [==============================] - 0s 114us/step - loss: 0.4243 - acc: 0.8267
Epoch 22/100
802/802 [==============================] - 0s 114us/step - loss: 0.4221 - acc: 0.8279
Epoch 23/100
802/802 [==============================] - 0s 118us/step - loss: 0.4202 - acc: 0.8292
Epoch 24/100
802/802 [==============================] - 0s 108us/step - loss: 0.4190 - acc: 0.8292
Epoch 25/100
802/802 [==============================] - 0s 120us/step - loss: 0.4175 - acc: 0.8279
Epoch 26/100
802/802 [==============================] - 0s 114us/step - loss: 0.4162 - acc: 0.8279
Epoch 27/100
802/802 [==============================] - 0s 113us/step - loss: 0.4141 - acc: 0.8292
Epoch 28/100
802/802 [==============================] - 0s 115us/step - loss: 0.4134 - acc: 0.8267
Epoch 29/100
802/802 [==============================] - 0s 116us/step - loss: 0.4116 - acc: 0.8267
Epoch 30/100
802/802 [=

802/802 [==============================] - 4s 5ms/step - loss: 0.6908 - acc: 0.6172
Epoch 2/100
802/802 [==============================] - 0s 150us/step - loss: 0.6844 - acc: 0.6147
Epoch 3/100
802/802 [==============================] - 0s 134us/step - loss: 0.6725 - acc: 0.6147
Epoch 4/100
802/802 [==============================] - 0s 126us/step - loss: 0.6528 - acc: 0.6185
Epoch 5/100
802/802 [==============================] - 0s 200us/step - loss: 0.6243 - acc: 0.7132
Epoch 6/100
802/802 [==============================] - 0s 109us/step - loss: 0.5890 - acc: 0.8055
Epoch 7/100
802/802 [==============================] - 0s 100us/step - loss: 0.5489 - acc: 0.8167
Epoch 8/100
802/802 [==============================] - 0s 96us/step - loss: 0.5119 - acc: 0.8192
Epoch 9/100
802/802 [==============================] - 0s 96us/step - loss: 0.4831 - acc: 0.8180
Epoch 10/100
802/802 [==============================] - 0s 102us/step - loss: 0.4622 - acc: 0.8217
Epoch 11/100
802/802 [=============

802/802 [==============================] - 0s 100us/step - loss: 0.3788 - acc: 0.8454
Epoch 85/100
802/802 [==============================] - 0s 106us/step - loss: 0.3790 - acc: 0.8466
Epoch 86/100
802/802 [==============================] - 0s 104us/step - loss: 0.3782 - acc: 0.8491
Epoch 87/100
802/802 [==============================] - 0s 112us/step - loss: 0.3786 - acc: 0.8454
Epoch 88/100
802/802 [==============================] - 0s 108us/step - loss: 0.3785 - acc: 0.8466
Epoch 89/100
802/802 [==============================] - 0s 129us/step - loss: 0.3778 - acc: 0.8454
Epoch 90/100
802/802 [==============================] - 0s 167us/step - loss: 0.3774 - acc: 0.8466
Epoch 91/100
802/802 [==============================] - 0s 125us/step - loss: 0.3777 - acc: 0.8392
Epoch 92/100
802/802 [==============================] - 0s 122us/step - loss: 0.3769 - acc: 0.8479
Epoch 93/100
802/802 [==============================] - 0s 123us/step - loss: 0.3768 - acc: 0.8454
Epoch 94/100
802/802 [=

In [12]:
print (mean,variance)

0.83168539293 0.0403300336015


In [13]:
classifier.fit(X_train, y_train)

Epoch 1/100
891/891 [==============================] - 4s 4ms/step - loss: 0.6902 - acc: 0.6162
Epoch 2/100
891/891 [==============================] - 0s 166us/step - loss: 0.6820 - acc: 0.6162
Epoch 3/100
891/891 [==============================] - 0s 162us/step - loss: 0.6648 - acc: 0.6184
Epoch 4/100
891/891 [==============================] - 0s 162us/step - loss: 0.6366 - acc: 0.7407
Epoch 5/100
891/891 [==============================] - 0s 166us/step - loss: 0.5960 - acc: 0.8114
Epoch 6/100
891/891 [==============================] - 0s 183us/step - loss: 0.5465 - acc: 0.8148
Epoch 7/100
891/891 [==============================] - 0s 165us/step - loss: 0.4972 - acc: 0.8182
Epoch 8/100
891/891 [==============================] - 0s 161us/step - loss: 0.4598 - acc: 0.8193
Epoch 9/100
891/891 [==============================] - 0s 141us/step - loss: 0.4386 - acc: 0.8204
Epoch 10/100
891/891 [==============================] - 0s 136us/step - loss: 0.4277 - acc: 0.8227
Epoch 11/100
891/891 

891/891 [==============================] - 0s 143us/step - loss: 0.3672 - acc: 0.8462
Epoch 84/100
891/891 [==============================] - 0s 116us/step - loss: 0.3667 - acc: 0.8474
Epoch 85/100
891/891 [==============================] - 0s 111us/step - loss: 0.3662 - acc: 0.8507
Epoch 86/100
891/891 [==============================] - 0s 115us/step - loss: 0.3663 - acc: 0.8474
Epoch 87/100
891/891 [==============================] - 0s 112us/step - loss: 0.3658 - acc: 0.8451
Epoch 88/100
891/891 [==============================] - 0s 119us/step - loss: 0.3653 - acc: 0.8519
Epoch 89/100
891/891 [==============================] - 0s 116us/step - loss: 0.3652 - acc: 0.8485
Epoch 90/100
891/891 [==============================] - 0s 116us/step - loss: 0.3648 - acc: 0.8462
Epoch 91/100
891/891 [==============================] - 0s 116us/step - loss: 0.3650 - acc: 0.8507
Epoch 92/100
891/891 [==============================] - 0s 113us/step - loss: 0.3641 - acc: 0.8507
Epoch 93/100
891/891 [=

In [14]:
X_test = test.iloc[:,[1,3,4,5,6,8,10,11,12,13,14,15]].values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
labelencoder_X1 = LabelEncoder()
X_test[:,11] =  labelencoder_X1.fit_transform(X_test[:,11])
onehotencoder = OneHotEncoder(categorical_features = [11])
X_test = onehotencoder.fit_transform(X_test).toarray()
X_test = X_test[:,1:]
onehotencoder_X2 = OneHotEncoder(categorical_features = [10])
X_test = onehotencoder_X2.fit_transform(X_test).toarray()
X_test = X_test[:,1:]
onehotencoder_X2 = OneHotEncoder(categorical_features = [6])
X_test = onehotencoder_X2.fit_transform(X_test).toarray()
X_test = X_test[:,1:]
from sklearn.preprocessing import StandardScaler 
sc= StandardScaler()
X_test=sc.fit_transform(X_test)

In [15]:
y_pred = classifier.predict(X_test)
df = pd.DataFrame(y_pred)
df.to_csv("C:\\Users\\hp-pc\\Desktop\\Deep Summer\\Titanic\\result12.csv")